# Site statique 1 : Wikipedia (extraire un tableau de données)

##  Installation des bibliothèques

**Ce projet vous guidera pour extraire des tableaux de données au format HTML**

In [1]:
#!pip install requests 
#!pip install bs4 import BeautifulSoup 
#!pip install pandas

##  Importation des bibliothèques

In [2]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import time #  module intégré dans Python
import re #  module intégré dans Python

##  Définir les en-têtes et le lien du site à scraper

In [3]:
# Les entêtes 'user-agent' sont spécifiques à votre système. Cela permet d'imiter un navigateur.
HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
}

# lien du site
lien = "https://en.wikipedia.org/wiki/List_of_largest_French_companies"

## Faire une requête et vérifier si elle a été exécutée avec succès

In [4]:
# Envoi de la requête GET à l'URL avec l'entête 'user-agent' défini
response = requests.get(lien, headers=HEADERS)
response.status_code

200


**response.status_code** retourne 200, ce qui signifie que la requête HTTP a été exécutée avec succès.

L'agent utilisateur **User-Agent** n'est pas toujours obligatoire, mais il est préférable de l'ajouter afin que le certain site ne vous bloque pas.

## Analyse de contenu HTML avec BeautifulSoup

In [6]:
# Analyse la réponse avec BeautifulSoup en utilisant l'analyseur HTML
soup = BeautifulSoup(response.text, "html.parser")

La variable **soup** contient l'object BeautifulSoup.

In [7]:
# prettify est une méthode pour affiche le contenu HTML formaté
print(soup.prettify()[:1500]) # affiche des 1500 premiers caratère)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of largest French companies - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature

## Sélection des balises et classe ou ID

Avec **BeautifulSoup** et ces méthodes **find()** et **find_all()**, vous pouvez récupérer des données spécifiques en passant la balise, ainsi que la classe ou l'ID comme paramètres pour cibler les données spécifiques.

* **find()**: renvoie le premier un élément HTML

* **find_all()**: renvoie la liste de tous les éléments HTML

In [8]:
# Recherche tous les tableaux avec la classe "wikitable sortable"
tables = soup.find_all("table", class_="wikitable sortable")

In [9]:
len(tables)

2

La variable **tables** contient une liste des tableaux HTML associés à la balise <table>

In [10]:
# Recherche des titres des sections d'années
annee = soup.find_all("div", class_="mw-heading mw-heading2")

# Récupération des années sous forme de liste
annees = [annee[i].find("h2").text for i in range(len(annee))]

La variable **annee** contient une liste de contenus HTML déterminés par la balise <div> et la classe "mw-heading mw-heading2".

In [11]:

# Liste des années extraites (seulement les chiffres)
rank_annees = [
    "".join(re.findall(r'\d+', annees[i])) for i in range(len(annees)) if "".join(re.findall(r'\d+', annees[i])) != ""
]
rank_annees

['2024', '2019']

**rank_annees** est une liste en compréhension qui extrait les années en utilisant des expressions régulières pour ne récupérer que les chiffres. Cette liste est équivalente au code suivant:

In [12]:
rank_years = []
for i in range(len(annees)):
    # Recupere juste les chifres dans la chaine de caractère
    year = "".join(re.findall(r'\d+', annees[i]))
    if year != "":
        rank_years.append(year)

rank_years

['2024', '2019']

In [13]:
# Liste pour stocker les DataFrames extraits des tableaux
liste_tableaux = []

# Boucle pour itérer à travers les tableaux et les années correspondantes
for table, ann in zip(tables, range(len(rank_annees))):
    # Extraction des entêtes de colonnes du tableau
    variables = table.find_all("th")

## 6. Récupérer les entêtes du tableau

In [14]:
# Liste en compréhension pour récupérer les en-têtes des tableaux
liste_variables = [variable.text.strip() for variable in variables]
liste_variables

['Rank',
 'Fortune 2000rank',
 'Name',
 'Headquarters',
 'Revenue(USD billions)',
 'Profits(USD billions)',
 'Total Assets(USD billions)',
 'Market Value(USD billions)',
 'Industry']

In [15]:
liste_variables[0]

'Rank'

In [16]:
# Modification du nom de la première colonne pour inclure l'année
liste_variables[0] = "Rank " + str(rank_annees[ann])
liste_variables[0]

'Rank 2019'

Le code suivant retourne toutes les lignes du tableau sans en-tête.

In [17]:
# Récupération des lignes de données, excluant la première ligne (les en-têtes)
lignes_data = table.find_all("tr")[1:]
# afficher la premiere ligne
lignes_data[0]

<tr>
<td>1
</td>
<td>25
</td>
<td align="left"><a class="mw-redirect" href="/wiki/Total_SA" title="Total SA">Total</a>
</td>
<td align="left">Courbevoie
</td>
<td>184.2
</td>
<td>11.4
</td>
<td>256.8
</td>
<td>149.5
</td>
<td align="left">Oil and gas
</td></tr>

In [18]:
# Liste pour stocker les lignes de données extraites
valeurs_lignes = []
for row in lignes_data:
    row_df = row.find_all("td")
    # Extraction des valeurs de chaque cellule de la ligne
    ligne = [content.text.strip() for content in row_df]
    valeurs_lignes.append(ligne)

# afficher les deux premiers lignes
valeurs_lignes[:2]

[['1',
  '25',
  'Total',
  'Courbevoie',
  '184.2',
  '11.4',
  '256.8',
  '149.5',
  'Oil and gas'],
 ['2',
  '34',
  'BNP Paribas',
  'Paris',
  '101.6',
  '8.4',
  '2,333.0',
  '68.7',
  'Banking']]

## 7. Mettre des données scrappées dans une dataframe

In [19]:
# Création d'un DataFrame avec les données extraites et les en-têtes de colonnes
df = pd.DataFrame(data=valeurs_lignes, columns=liste_variables)
df.head()

,Rank 2019,Fortune 2000rank,Name,Headquarters,Revenue(USD billions),Profits(USD billions),Total Assets(USD billions),Market Value(USD billions),Industry
0,1,25,Total,Courbevoie,184.2,11.4,256.8,149.5,Oil and gas
1,2,34,BNP Paribas,Paris,101.6,8.4,"2,333.0",68.7,Banking
2,3,85,Axa,Paris,139.7,2.2,"1,034.5",63.6,Insurance
3,4,104,Crédit Agricole,Paris,52.2,4.7,"1,856.9",38.4,Banking
4,5,114,Sanofi,Paris,40.7,5.1,127.4,102.0,Pharmaceuticals


## 8. Sauvegardez les données dans un fichier

In [20]:
# Sauvegarde des données dans un fichier CSV
df.to_csv(f"List_of_largest_French_companies-{rank_annees[ann]}.csv", index=False)

In [21]:
# Sauvegarde des données dans un fichier Excel
# df.to_excel(f"List_of_largest_French_companies-{rank_annees[ann]}.xlsx", index=False)

In [22]:
# Création d'un DataFrame avec les données extraites et les en-têtes de colonnes
df = pd.DataFrame(data=valeurs_lignes, columns=liste_variables)
df.head()

,Rank 2019,Fortune 2000rank,Name,Headquarters,Revenue(USD billions),Profits(USD billions),Total Assets(USD billions),Market Value(USD billions),Industry
0,1,25,Total,Courbevoie,184.2,11.4,256.8,149.5,Oil and gas
1,2,34,BNP Paribas,Paris,101.6,8.4,"2,333.0",68.7,Banking
2,3,85,Axa,Paris,139.7,2.2,"1,034.5",63.6,Insurance
3,4,104,Crédit Agricole,Paris,52.2,4.7,"1,856.9",38.4,Banking
4,5,114,Sanofi,Paris,40.7,5.1,127.4,102.0,Pharmaceuticals


In [23]:

# Sauvegarde des données dans un fichier CSV ou Excel
df.to_csv(f"List_of_largest_French_companies-{rank_annees[ann]}.csv", index=False)

# df.to_excel(f"List_of_largest_French_companies-{rank_annees[ann]}.xlsx", index=False)

=================================================================================================================================================

# Méthode pour récupérer des tableaux de données sur Wikipedia

In [24]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import time #  module intégré dans Python
import re #  module intégré dans Python


# Les entêtes 'user-agent' sont spécifiques à votre système. Cela permet d'imiter un navigateur.
HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
}


def scrape_tableau(url):
    """  
    Méthode pour scraper des tableaux d'une page web.
    Args:
        url (str): Lien de la page à scraper.
    Retourne: Une liste de DataFrames, chaque DataFrame représentant un tableau extrait de la page.
    """
    # Envoi de la requête GET à l'URL avec l'entête 'user-agent' défini
    response = requests.get(url, headers=HEADERS)
    
    # Pause pour ne pas surcharger le serveur avec des requêtes consécutives
    time.sleep(3)
    
    # Vérifie si la requête a réussi
    if response.status_code == 200:
        # Analyse la réponse avec BeautifulSoup en utilisant l'analyseur HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Recherche tous les tableaux avec la classe "wikitable sortable"
        tables = soup.find_all("table", class_="wikitable sortable")
        
        # Recherche des titres des sections d'années
        annee = soup.find_all("div", class_="mw-heading mw-heading2")
        
        # Récupération des années sous forme de liste
        annees = [annee[i].find("h2").text for i in range(len(annee))]
        
        # Liste des années extraites (seulement les chiffres)
        rank_annees = ["".join(re.findall(r'\d+', annees[i])) for i in range(len(annees))
                        if "".join(re.findall(r'\d+', annees[i])) != ""]
        
        # Liste pour stocker les DataFrames extraits des tableaux
        liste_tableaux = []

        # Boucle pour itérer à travers les tableaux et les années correspondantes
        for table, ann in zip(tables, range(len(rank_annees))):
            # Extraction des entêtes de colonnes du tableau
            variables = table.find_all("th")

            # Liste en compréhension pour récupérer les en-têtes des tableaux
            liste_variables = [variable.text.strip() for variable in variables]
            
            # Modification du nom de la première colonne pour inclure l'année
            liste_variables[0] = "Rank of " + str(rank_annees[ann])

            # Récupération des lignes de données, excluant la première ligne (les en-têtes)
            lignes_data = table.find_all("tr")[1:]

            # Liste pour stocker les lignes de données extraites
            valeurs_lignes = []
            for row in lignes_data:
                row_df = row.find_all("td")
                # Extraction des valeurs de chaque cellule de la ligne
                ligne = [content.text.strip() for content in row_df]
                valeurs_lignes.append(ligne)

            # Création d'un DataFrame avec les données extraites et les en-têtes de colonnes
            df = pd.DataFrame(data=valeurs_lignes, columns=liste_variables)

            # Sauvegarde des données dans un fichier CSV ou Excel
            df.to_csv(f"List_of_largest_French_companies-{rank_annees[ann]}.csv", index=False)
            # Vous pouvez aussi sauvegarder en format Excel en décommentant la ligne suivante:
            # df.to_excel(f"List_of_largest_French_companies-{rank_annees[ann]}.xlsx", index=False)
            
            # Ajout du DataFrame à la liste
            liste_tableaux.append(df)

    # Retourne la liste de DataFrames
    return liste_tableaux

In [25]:
# Le lien dans la variable "url" pointe vers une page Wikipedia contenant deux tableaux de données.
# La fonction 'scrape_tableau' est définie pour récupérer ces deux tableaux et les stocker dans une liste.

url = "https://en.wikipedia.org/wiki/List_of_largest_French_companies" 

# Appel de la fonction 'scrape_tableau' avec l'URL fournie pour récupérer les tableaux.
liste_tableaux = scrape_tableau(url)

In [26]:

# Affichage du premier tableau extrait
df0 = liste_tableaux[0]

# Affichage du 5 premiers lignes du dataframe
df0.head(5)

,Rank of 2024,Fortune 500rank,Name,Industry,Revenue(USD millions),Profits(USD millions),Employees,Headquarters
0,1,20,TotalEnergies,Oil and gas,"218,945","21,384","102,579",Courbevoie
1,2,49,Électricité de France,Electric utility,"151,040","10,828","171,862",Paris
2,3,64,BNP Paribas,Banking,"136,073","11,864","182,656",Paris
3,4,104,Société Générale,Banking,"99,163","2,695","124,089",Paris
4,5,118,Crédit Agricole,Banking,"93,358","6,863","75,125",Paris


In [27]:
df1 = liste_tableaux[1]
df1.head()

,Rank of 2019,Fortune 2000rank,Name,Headquarters,Revenue(USD billions),Profits(USD billions),Total Assets(USD billions),Market Value(USD billions),Industry
0,1,25,Total,Courbevoie,184.2,11.4,256.8,149.5,Oil and gas
1,2,34,BNP Paribas,Paris,101.6,8.4,"2,333.0",68.7,Banking
2,3,85,Axa,Paris,139.7,2.2,"1,034.5",63.6,Insurance
3,4,104,Crédit Agricole,Paris,52.2,4.7,"1,856.9",38.4,Banking
4,5,114,Sanofi,Paris,40.7,5.1,127.4,102.0,Pharmaceuticals
